## Scrape the EWRIMS Database

A walk through for scraping the entire EWRIMS database. This module allows for appending new applications and reports to existing datasets. This module was developed in consultation with the Center for Environmental Economics and Sustainability Policy at the W.P. Carey School of Business under the direction of Dr. William Michael Hanemann.

* **Applications:** each application is an application to the water rights for a specific body of water. Applications may detail beneficial water use among other pertinant details. Applications may be submitted multiple times over time as required by local authorities, private contracts, or others. The state of California does not penalize a failure to apply for water rights so this database is not a complete representation of water use in the state. <br><br>

* **Reports:** reports may be submitted in association with particular applications. Reports cover a wide vireity of topics and may describe anything from the amount of water used over time to conservations efforts to changes to the project to methods of measurement. 

In [1]:
from importlib import reload
from datetime import datetime

import os 
import numpy as np 
import pandas as pd
import bear_necessities as bn 
import ewrmis_master_data as ewd

from importlib import reload
ewd = reload(ewd)

from ewrmis_master_data import *

*Scrape the all the applications currently hosted in the database*
599

In [2]:
import time 
update_complete = False

while update_complete==False: 
    try: 
        application_data = ewd.scrape_applications()
        update_complete = True
        print('Update Completed')
    except Exception as e: 
        print(str(e))
        # if we don't error out due to time 
        if 'timeout' not in str(e): 
            update_complete=True 
        else: 
        # if we timed out take a break for 2 seconds and retry
            time.sleep(2)

Message: chrome not reachable
  (Session info: chrome=75.0.3770.142)



In [42]:
application_data = bn.loosen(os.getcwd()+'/data/database/database/master_data.pickle')
# drop duplicate applications
application_data = application_data.drop_duplicates(subset='ApplID')
# remove spaces from the column names 
application_data.columns = [c.replace(' ','') for c in application_data.columns]

# format the dates in the scraped application data into datetime format 
application_data.loc[application_data['Date']!='','Date'] = application_data.loc[application_data['Date']!='','Date'].apply(lambda s:datetime.strptime(s, '%m/%d/%Y'))
len(application_data)

58169

*Load an existing dataset with application data*

In [67]:
application_master = pd.read_csv(os.getcwd()+'/data/database/database/master_data.csv')
# format the dates in the older application data into datetime format 
application_master.loc[application_master['date'].notnull(),'date']= application_master.loc[application_master['date'].notnull(),'date'].apply(lambda s:datetime.strptime(s, '%m/%d/%Y'))
len(application_master)

55287

In [50]:
new_max = application_data.loc[application_data['Date']!='','Date'].max()
old_max = application_master.loc[application_master['date'].notnull(),'date'].max()
print("Most recent date from newest scrape: ", new_max)
print("Most recent date from oldest scrape: ", old_max)

Most recent date from newest scrape:  2019-07-18 00:00:00
Most recent date from oldest scrape:  2018-09-20 00:00:00


*Check how many applications have been submitted (new) or modifieid since the last scrape*

In [57]:
new_or_modified = application_data.loc[application_data['Date']!=''].loc[application_data.loc[application_data['Date']!='','Date']>old_max]
print("# of new or modified applications since last date: ", str(len(new_or_modified)))

# of new or modified applications since last date:  1780


*Check how many new application IDs have been submitted*

In [69]:
new_applications = np.setdiff1d(application_data['ApplID'].values, application_master['ApplID'].values)
print('New Applications: ',str(len(new_applications)))

new_applications = application_data.set_index('ApplID').loc[new_applications].reset_index()
print('New Applications Records: ',str(len(new_applications)))

New Applications:  1537
New Applications Records:  2119


In [76]:
# Check how many unique new or modified applications. 
apps = new_applications.append(new_or_modified, ignore_index=True, sort=False)
print(len(apps.drop_duplicates()))
print(len(apps.drop_duplicates(subset='ApplID')))

# We will scrape the application and reports for unique new or modified applications 
apps = apps.drop_duplicates(subset='ApplID')

1966
1964


*This function automatically creates a ".pickle" file in the database for each of these tables for each 100 scraped and upon finishing. It keeps track of progress using the `current_application.txt` file in the runtime folder. Set the value in the .txt file to 0 to restart the scraping from the 0th row in the submitted application data.* 

In [77]:
# Get the for every application
summary_data, current_parties, histparties, record_summary, sources, uses, report, decisions = scrape_application_details(apps)

*We can load the report links so we can scrape the details from each report*

In [2]:
reports = bn.loosen(os.getcwd()+'/data/database/database/electronic_reports.pickle')
reports['ViewReportPDF'] = reports['ViewReportPDF'].apply(lambda x: x.replace('..','https://ciwqs.waterboards.ca.gov/ciwqs'))

*Here we scrape the links from each report*

In [6]:
links = list(reports['ViewReportPDF'].values)
ids = list(reports['ApplID'].values)
datasets = scrape_reports(links, ids)